# Synthetic Data on Healthcare

This notebook is a demonstration on how to use SQLSynthGen for generating synthetic data, for a healthcare use case.

## Initial Setup

In [2]:
from os import chdir

HOME_DIRECTORY: str = "/home/jovyan"

chdir(HOME_DIRECTORY)

from lib import healthcare_demo

## Check the data

Let's take a look at what's stored in the database. The `dataset` column has a value of `1` for patients with liver disease, and a value of `2` for no disease.

In [14]:
import pandas as pd

DATABASE_NAME: str = "db"

connection, _ = healthcare_demo.connect_to_postgress(DATABASE_NAME)
query_dataframe = pd.read_sql_query(
    "SELECT * FROM indian_liver_patient", con=connection
)

query_dataframe

/tmp/ipykernel_496/3251272698.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  query_dataframe = pd.read_sql_query(


,age,gender,total_bilirubin,direct_bilirubin,alkaline_phosphotase,alamine_aminotransferase,aspartate_aminotransferase,total_protiens,albumin,albumin_and_globulin_ratio,dataset,id
0,65,Female,0.7,0.1,187,16,18,6.8,3.3,0.90,1,1
1,62,Male,10.9,5.5,699,64,100,7.5,3.2,0.74,1,2
2,62,Male,7.3,4.1,490,60,68,7.0,3.3,0.89,1,3
3,58,Male,1.0,0.4,182,14,20,6.8,3.4,1.00,1,4
4,72,Male,3.9,2.0,195,27,59,7.3,2.4,0.40,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...
578,60,Male,0.5,0.1,500,20,34,5.9,1.6,0.37,2,579
579,40,Male,0.6,0.1,98,35,31,6.0,3.2,1.10,1,580
580,52,Male,0.8,0.2,245,48,49,6.4,3.2,1.00,1,581
581,31,Male,1.3,0.5,184,29,32,6.8,3.4,1.00,1,582


In [23]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,):
    print(query_dataframe[["age", "gender"]].value_counts())

age  gender
60   Male      33
50   Male      18
32   Male      18
45   Male      16
33   Male      15
38   Male      15
55   Male      14
65   Male      13
42   Male      13
40   Male      13
75   Male      12
26   Male      11
48   Male      11
46   Male      11
18   Male      10
51   Male       9
62   Male       9
58   Male       9
66   Male       9
45   Female     9
48   Female     9
35   Male       8
36   Male       8
49   Male       8
37   Male       8
42   Female     8
72   Male       8
70   Male       7
57   Male       7
30   Male       6
52   Male       6
27   Male       6
39   Male       6
38   Female     6
22   Male       6
34   Male       6
21   Male       6
31   Male       5
50   Female     5
46   Female     5
64   Male       5
61   Male       5
58   Female     5
54   Male       4
47   Male       4
41   Male       4
55   Female     4
40   Female     4
28   Female     4
     Male       4
54   Female     4
65   Female     4
35   Female     4
29   Male       4
30   Female     

In [25]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,):
    print(query_dataframe["age"].value_counts())

age
60    34
45    25
50    23
42    21
38    21
32    20
48    20
55    18
65    17
40    17
46    16
33    15
58    14
75    14
26    14
35    12
66    12
18    11
49    11
36    11
51    10
30    10
62     9
70     9
22     9
37     9
34     8
28     8
72     8
31     8
54     8
52     7
57     7
21     7
29     7
53     6
39     6
27     6
64     6
47     6
41     5
61     5
24     5
25     5
17     5
56     4
43     4
13     4
68     4
74     4
16     3
44     3
23     3
20     3
63     2
14     2
73     2
69     2
12     2
4      2
7      2
19     2
6      1
8      1
84     1
85     1
15     1
78     1
11     1
67     1
10     1
90     1
Name: count, dtype: int64


We see 10 people have 51 years, but only 1 is female